In [1]:
%matplotlib notebook
#import matplotlib.pyplot as plt
# Note! ITK interacts weirdly here.  from lazy_imports import itk does not work.
# Additionally, import itk must occur before lazy_imports for itkwidgets.view (ie itkview) to work.
import itk
#from lazy_imports import itk
#from lazy_imports import torch
import torch # to avoid sym3eig_cpu ImportError: libc10.so: cannot open shared object file: No such file or directory
from lazy_imports import np
from lazy_imports import plt
from lazy_imports import sitk
from lazy_imports import loadmat, savemat
from lazy_imports import sitk
from lazy_imports import itkwidgets
from lazy_imports import itkview
from lazy_imports import interactive
from lazy_imports import ipywidgets
from lazy_imports import pv
#from lazy_imports import vlinalg
from torch_sym3eig import Sym3Eig
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)

In [2]:
from mtch.SplitEbinMetric3D import get_karcher_mean

In [3]:
from util.diffeo import get_idty_3d, get_gradient_3d, compose_function_3d

In [4]:
from mtch.metricMatching import metric_matching

In [5]:
from disp.vis import show_2d, show_2d_tensors
from disp.vis import vis_tensors, vis_path
from disp.vis import view_3d_tensors, tensors_to_mesh, view_3d_paths, path_to_tube
from data.io import readRaw, ReadScalars, ReadTensors, WriteTensorNPArray, WriteScalarNPArray, readPath3D
from data.convert import GetNPArrayFromSITK, GetSITKImageFromNP

In [12]:
do_GPU=False
#do_GPU=True

use_brain=True
#use_brain=False

if do_GPU:
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
  torch.cuda.set_device(1)
else:
  torch.set_default_tensor_type('torch.FloatTensor')  


In [20]:
if use_brain:
  cases = []
  #all_start_coords = []
  cases.append('103818')
  #all_start_coords.append([[62,126,56]])
  cases.append('105923')
  #all_start_coords.append([[61,125,56]])
  #cases.append('111312')
  #all_start_coords.append([[62,128,56]])
  masks = []
  t1s = []
  in_tensors = []
  out_tensors = []
  outdirs = []
  for run_case in cases:
    subj = run_case[:6]
    #outroot='/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python/simulation_results/'
    #outdir = f'{outroot}{run_case}/mineval_0.005_n_3000_s_1.5/'
    #indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_data/' + subj + '/'

    outroot='/Volumes/GoogleDrive/Shared\ drives/NSFCRCNSHumanConectome/Data/3D_results/'
    outdir = f'{outroot}{run_case}/'
    indir = '/Volumes/GoogleDrive/Shared\ drives/NSFCRCNSHumanConectome/Data/prepped_data/' + subj + '/'

    #bval = 3000
    #outroot='/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python/UKF_experiments/'
    #indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_UKF_data_with_grad-dev/' + subj + '/'
    bval = 1000
    
    outdirs.append(outdir)

    tens_file = f'dti_{bval}_tensor.nhdr'
    in_tens = ReadTensors(indir + tens_file)
    tens_file = 'scaled_tensors.nhdr'
    out_tens = ReadTensors(outdir + tens_file)
    #t1_file = 't1_stripped_irescaled.nhdr'
    #t1 = ReadScalars(indir + t1_file)
    mask_file = 'filt_mask.nhdr'
    mask = ReadScalars(outdir + mask_file)
    in_tensors.append(in_tens)
    out_tensors.append(out_tens)
    #t1s.append(t1)
    masks.append(mask)
    #t1_flip = ReadScalars(indir + 't1_rescaled_flipped.nhdr')[:,::-1,:]


  #atlasdir = '/home/sci/hdai/Shared/BrainAtlasMay21/'
  #atlas_tens = ReadTensors(atlasdir + 'atlas_tens.nhdr')
  #atlas_mask = ReadScalars(atlasdir + 'atlas_mask.nhdr')
  atlasdir = '/home/sci/hdai/Shared/Brain3AtlasMay24/'
  atlas_tens = ReadTensors(atlasdir + 'atlas_800_tens.nhdr')
  atlas_mask = ReadScalars(atlasdir + 'atlas_800_mask.nhdr')

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:99:
sitk::ERROR: The file "/Volumes/GoogleDrive/Shared\ drives/NSFCRCNSHumanConectome/Data/prepped_data/103818/dti_1000_tensor.nhdr" does not exist.

In [14]:
%%time
if use_brain:
  tens_0 = np.zeros((out_tensors[0].shape[0],out_tensors[0].shape[1],out_tensors[0].shape[2],3,3))
  tens_1 = np.zeros((out_tensors[1].shape[0],out_tensors[1].shape[1],out_tensors[1].shape[2],3,3))
  for xx in range(tens_0.shape[0]):
    for yy in range(tens_0.shape[1]):
      for zz in range(tens_0.shape[2]):
        if masks[0][xx,yy,zz]:
          tens_0[xx,yy,zz,0,0] = out_tensors[0][xx,yy,zz,0]
          tens_0[xx,yy,zz,0,1] = out_tensors[0][xx,yy,zz,1]
          tens_0[xx,yy,zz,1,0] = out_tensors[0][xx,yy,zz,1]
          tens_0[xx,yy,zz,0,2] = out_tensors[0][xx,yy,zz,2]
          tens_0[xx,yy,zz,2,0] = out_tensors[0][xx,yy,zz,2]
          tens_0[xx,yy,zz,1,1] = out_tensors[0][xx,yy,zz,3]
          tens_0[xx,yy,zz,1,2] = out_tensors[0][xx,yy,zz,4]
          tens_0[xx,yy,zz,2,1] = out_tensors[0][xx,yy,zz,4]
          tens_0[xx,yy,zz,2,2] = out_tensors[0][xx,yy,zz,5]
        else:
          tens_0[xx,yy,zz,0,0] = 1
          tens_0[xx,yy,zz,1,1] = 1
          tens_0[xx,yy,zz,2,2] = 1
        if masks[1][xx,yy,zz]:
          tens_1[xx,yy,zz,0,0] = out_tensors[1][xx,yy,zz,0]
          tens_1[xx,yy,zz,0,1] = out_tensors[1][xx,yy,zz,1]
          tens_1[xx,yy,zz,1,0] = out_tensors[1][xx,yy,zz,1]
          tens_1[xx,yy,zz,0,2] = out_tensors[1][xx,yy,zz,2]
          tens_1[xx,yy,zz,2,0] = out_tensors[1][xx,yy,zz,2]
          tens_1[xx,yy,zz,1,1] = out_tensors[1][xx,yy,zz,3]
          tens_1[xx,yy,zz,1,2] = out_tensors[1][xx,yy,zz,4]
          tens_1[xx,yy,zz,2,1] = out_tensors[1][xx,yy,zz,4]
          tens_1[xx,yy,zz,2,2] = out_tensors[1][xx,yy,zz,5]
        else:
          tens_1[xx,yy,zz,0,0] = 1
          tens_1[xx,yy,zz,1,1] = 1
          tens_1[xx,yy,zz,2,2] = 1
        

  if do_GPU:
    torch_tens_0 = torch.from_numpy(tens_0).cuda().float()
    torch_tens_1 = torch.from_numpy(tens_1).cuda().float()
    mask_union = torch.from_numpy(masks[0]).cuda().float()
    mask_union += torch.from_numpy(masks[1]).cuda().float()

  else:
    torch_tens_0 = torch.from_numpy(tens_0).float()
    torch_tens_1 = torch.from_numpy(tens_1).float()
    mask_union = torch.from_numpy(masks[0]).float()
    mask_union += torch.from_numpy(masks[1]).float()
    

CPU times: user 13 s, sys: 236 ms, total: 13.2 s
Wall time: 12.8 s


In [15]:
%%time
#use_idty=True
use_idty=False
dim, sigma, epsilon, iter_num = 3., 0, 5e-3, 3
height, width, depth = 145,174,145
if not use_brain:
  height, width, depth = 20,30,40
  a = torch.rand((20,30,40, 3, 3))
  a = 0.5 * (a + a.transpose(3, 4))
  b = torch.rand((20,30,40, 3, 3))
  b = 0.5 * (b + b.transpose(3, 4))

#with torch.autograd.detect_anomaly():
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  if use_brain:    
    tens_0_def, phi_0, phi_inv_0 = metric_matching(torch.inverse(torch_tens_0), 
                                                   torch.inverse(torch_tens_1), 
                                                   height, width, depth, mask_union, iter_num, epsilon, sigma,dim,use_idty)
  else:
    a_def, phi_a, phi_inv_a = metric_matching(a, b, 
                                              height, width, depth, torch.ones((height,width,depth)), iter_num, epsilon, sigma,dim,use_idty)
print(prof.key_averages().table(sort_by="self_cpu_time_total"))
# Torch CPU 3 iters: CPU time: 7 min 17 s, Wall time: 2 min 7s
# Torch CPU 3 iters: CPU time: 4min 7 s, Wall time: 1 min 39s
# Torch GPU w/ anomaly detection 3 iters: CPU time: 40 min 20s, Wall time: 38 min 7s
# Torch GPU 3 iters: CPU time: 39min 38s, Wall time: 37min 55s



0 562705.8125
1 537092.375
2 557556.75
-----------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                               Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-----------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
               aten::_symeig_helper        60.06%       49.419s        60.28%       49.597s        8.266s       49.419s        59.99%       49.598s        8.266s             6  
              aten::_inverse_helper         8.96%        7.369s         9.56%        7.865s     393.263ms        7.369s         8.95%        7.867s     393.328ms            20  
             aten::triangular_solve         6.34%        5.214s       

In [1]:
#del tens_0_def_GPU, phi_0_GPU, phi_inv_0_GPU
#del tens_0_def, phi_0, phi_inv_0
#torch.cuda.empty_cache()

In [2]:
%%time
#a = torch.rand(5000, 3, 3,dtype=torch.double)
#a = 0.5 * (a + a.transpose(1, 2))
a = torch.rand(145,174,145, 3, 3,dtype=torch.double)
a = 0.5 * (a + a.transpose(3, 4))
w, wv = torch.symeig(a,True)  # fast (~0.0006s)


NameError: name 'torch' is not defined

In [205]:
%%time
t, tv = vlinalg.vSymEig(a.permute((1,2,0)).reshape((1,9,-1,1,1)),True)
e = t.reshape((3,-1)).permute((1,0))
ev = tv.reshape((3,3,-1)).permute((2,0,1))

CPU times: user 9.86 ms, sys: 0 ns, total: 9.86 ms
Wall time: 6.84 ms


In [207]:
wediff = torch.abs(w-e)
print(torch.min(w), torch.min(e))
print(torch.min(wediff), torch.max(wediff))
awv = torch.einsum('...ij,...jk->...ik',a,wv)
wwv = torch.einsum('...i,...ji->...ji',w,wv)
print(torch.max(torch.abs(awv-wwv)))
aev = torch.einsum('...ij,...jk->...ik',a,ev.double())
eev = torch.einsum('...i,...ji->...ji',e,ev)
print(torch.max(torch.abs(aev-eev)))


tensor(-1.0677, dtype=torch.float64) tensor(-1.0677)
tensor(5.3951e-13, dtype=torch.float64) tensor(1.1847e-07, dtype=torch.float64)
tensor(1.7764e-15, dtype=torch.float64)
tensor(1.7419e-07, dtype=torch.float64)


In [10]:
%%time
s3, s3v = Sym3Eig.apply(a.reshape((-1,3,3)))

CPU times: user 1.68 s, sys: 93.8 ms, total: 1.78 s
Wall time: 1.53 s


In [212]:
ws3diff = torch.abs(torch.sort(w)[0]-torch.sort(s3)[0])
print(torch.min(w), torch.min(s3))
print(torch.min(ws3diff), torch.max(ws3diff))
es3diff = torch.abs(e-s3)
print(torch.min(es3diff), torch.max(es3diff))
as3v = torch.einsum('...ij,...jk->...ik',a,s3v.double())
s3s3v = torch.einsum('...i,...ji->...ji',s3,s3v)
print(torch.max(torch.abs(as3v-s3s3v)))


tensor(-1.0677, dtype=torch.float64) tensor(-1.0677, dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(2.9365e-14, dtype=torch.float64)
tensor(5.3924e-13, dtype=torch.float64) tensor(2.9153, dtype=torch.float64)
tensor(4.4846e-14, dtype=torch.float64)


In [110]:
%%time
a = a.cuda()
w, _ = torch.symeig(a)  # slow (~0.9s)

CPU times: user 1.06 s, sys: 7.75 s, total: 8.81 s
Wall time: 8.82 s


In [111]:
%%time
e = vlinalg.vSymEig(a.permute((1,2,0)).reshape(1,9,-1,1,1))[0].reshape((3,-1)).permute((1,0))

CPU times: user 4.7 ms, sys: 106 µs, total: 4.81 ms
Wall time: 3.59 ms


In [112]:
wediff = torch.abs(w-e)
print(torch.min(wediff), torch.max(wediff))

tensor(4.4409e-14, device='cuda:0', dtype=torch.float64) tensor(1.1909e-07, device='cuda:0', dtype=torch.float64)


In [116]:
%%time
s3, _ = Sym3Eig.apply(a)

CPU times: user 2.33 ms, sys: 288 µs, total: 2.61 ms
Wall time: 1.75 ms


In [117]:
ws3diff = torch.abs(w-s3)
print(torch.min(ws3diff), torch.max(ws3diff))
es3diff = torch.abs(e-s3)
print(torch.min(es3diff), torch.max(es3diff))

tensor(0., device='cuda:0', dtype=torch.float64) tensor(2.9406, device='cuda:0', dtype=torch.float64)
tensor(4.5131e-14, device='cuda:0', dtype=torch.float64) tensor(2.9406, device='cuda:0', dtype=torch.float64)


In [213]:
print(a.shape)

torch.Size([5000, 3, 3])
